<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/ac-willeke/urban-climate"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" style="filter: invert(100%)"/> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/NINAnor/mapper-soilCondition/blob/main/src/data/GEE_display_data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# ArcGIS Pro | Calculate Canopy Fraction

**Author**: Willeke A'Campo

**Description:** This notebooks can be opend in ArcGIS Pro and used to calculate the canopy fraction per 100x100m grid cell. 

The following steps are performed:
1. Open the notebook in ArcGIS Pro and set the input and output paths.

2. Create a polygon grid of 100x100m based on the Land Surface Temperature raster extracted from GEE. 

3. Rasterize the tree crown polygons to a 2x2m raster and set the value of each cel to its area in m2.
    - If pixel is 100% covered by a tree crown, the value of the pixel is 4m2.
    - `Con(IsNull("raster"), 0, 4)`

4. Sum the values of the 2x2m raster per 100x100m grid cell. (Zonal Statistics)

5. Resample the 2x2m canopy raster to 100x100m. (Resample)

6. Calculate the Canopy Fraction per 100m grid cel by dividing the canopy raster values by 10000 m2 (Raster Calculator).


In [ ]:
import arcpy 
# step 1
reference_raster = "path/to/LSTraster"
fc_grid = "path/to/grid"
r_area_per_cell = "path/to/area_per_cell"
r_canopy_fraction_2m = "path/to/canopy_fraction_2m"
r_canopy_fraction_100m = "path/to/canopy_fraction_100m"

# Step 2
# use the reference raster to create a fishnet
arcpy.management.CreateFishnet(
    out_feature_class= fc_grid,
    origin_coord="438884.999999274 6444514.99987627",
    y_axis_coord="438884.999999274 6444524.99987627",
    cell_width=100,
    cell_height=100,
    number_rows=None,
    number_columns=None,
    corner_coord="443084.999999309 6447814.9998763",
    labels="LABELS",
    template='438884.999999274 6444514.99987627 443084.999999309 6447814.9998763 PROJCS["ETRS_1989_UTM_Zone_32N",GEOGCS["GCS_ETRS_1989",DATUM["D_ETRS_1989",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",9.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    geometry_type="POLYGON"
)

# step 3 - calculate area per cell
out_raster = arcpy.ia.ZonalStatistics(
    in_zone_data="grid_100m",
    zone_field="FID",
    in_value_raster="area_per_cell_4m",
    statistics_type="SUM",
    ignore_nodata="DATA",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_value=90,
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)
out_raster.save(r_area_per_cell)

# 
with arcpy.EnvManager(snapRaster=reference_raster):
    arcpy.management.Resample(
        in_raster="total_area_bodo",
        out_raster=r_area_per_cell,
        cell_size=100100,
        resampling_type="NEAREST"
    )
    
# step 6 - calculate canopy fraction and resample to 100x100m  
output_raster = arcpy.ia.RasterCalculator(
    expression=' "total_area_canopy.tif" /10000'
)
output_raster.save(r_canopy_fraction_2m)

with arcpy.EnvManager(outputCoordinateSystem='PROJCS["ETRS_1989_UTM_Zone_32N",GEOGCS["GCS_ETRS_1989",DATUM["D_ETRS_1989",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",9.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', snapRaster="kristiansand_raster_stack_100m_25832.tif_DTM", extent='438884.999999274 6444514.99987627 443084.999999309 6447814.9998763 PROJCS["ETRS_1989_UTM_Zone_32N",GEOGCS["GCS_ETRS_1989",DATUM["D_ETRS_1989",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",9.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'):
    arcpy.management.Resample(
        in_raster=r_canopy_fraction_2m
        out_raster=r_canopy_fraction_100m,
        cell_size="100.000000000838 100.000000000847",
        resampling_type="NEAREST"
    )

# step 7  optional expand raster by one cell
out_raster = arcpy.sa.Expand(
    in_raster=r_canopy_fraction_100m,
    number_cells=1,
    zone_values=[0],
    expand_method="MORPHOLOGICAL"
)
out_raster.save(r_canopy_fraction_100m_expanded)